In [1]:
import h3
import json
from shapely.geometry import Polygon, mapping

In [2]:
# Load the GeoJSON file
with open('bbox.geojson', 'r') as file:
    bounding_box_geojson = json.load(file)

In [3]:
def hex_to_geojson(h):
    boundary = h3.h3_to_geo_boundary(h, True)
    polygon = Polygon(boundary)
    return mapping(polygon)

def generate_geojson_for_resolution(resolution, bounding_box_geojson):
    # Extract the first feature's geometry (polygon) from the GeoJSON
    bounding_polygon_geojson = bounding_box_geojson['features'][0]['geometry']
    
    # Generate hexagons for the bounding polygon at the specified resolution
    hexagons = h3.polyfill(bounding_polygon_geojson, resolution, geo_json_conformant=True)
    
    features = []
    for hexagon in hexagons:
        geometry = hex_to_geojson(hexagon)
        
        # Get the centroid (location) of the hexagon
        location = h3.h3_to_geo(hexagon)
        
        # Get the neighbors of the hexagon
        neighbors = h3.k_ring(hexagon, 1)
        
        # Add index, neighbors, and location as properties
        properties = {
            "index": hexagon,
            #"neighbors": [str(neighbor) for neighbor in neighbors if neighbor != hexagon],
            "location": location
        }
        
        features.append({'type': 'Feature', 'geometry': geometry, 'properties': properties})
    
    geojson = {'type': 'FeatureCollection', 'features': features}
    
    with open(f'resolution_{resolution}_itajai.geojson', 'w') as file:
        json.dump(geojson, file)


In [4]:
# Generate GeoJSON for resolutions 10 to 15
for resolution in range(5, 13):
    generate_geojson_for_resolution(resolution, bounding_box_geojson)